# Code Workflows and Tooling for Research

## My experience: a balancing act

<center><img src="images/ss_shipit.jpg" width="500"></center>

- "getting things done" vs. "doing things the right way"
- improving workflows is a process

- "just ship it" mentality
- "technical debt" analogy: the uncertainty of research makes taking on technical debt that much easier
- try to start building habits that make "doing things the right way" efficient
- it is a process: don't expect yourself to change all at once, make incremental improvements in workflow
- also about building the right habits
- will go through some not so great examples (from yours truly), some easy things to try and incorporate into your workflow, as well as some more involved setups

## Outline

- Documentation
- Version Control
- Testing
- Automation
- Reproducibility


## Documentation

1. Write comments as if other people will read it
2. Show, don't tell
3. Consider documentation generators

- TODO reference the ten simple rules for documentation paper

## Version Control

1. Commit messages should be informative
2. Ideally subdivide tasks into concrete commits
3. Consider branching strategies

### Commit Messages

### Consider Development Branches

## Testing

1. Think about what your code "should do"
2. Ideally write dedicated tests
3. Consider testing suites

### Test-driven development mindset 

In [1]:
# TODO example of writing unit tests first

### `assert` statements

### Consider `unittest` and code coverage

## Automation

1. Move your workflows out of "interactive mode" as soon as possible
2. Ideally batch process computation
3. Consider workflow automation tools like `make`

### Jupyter notebooks are __notebooks__

- good for playing with the data
- good for presenting final results
- bad for "doing work" in between

- note that "doing work" may never happen on a particular research thread, hence the appeal of Jupyter notebooks for research

### `argparse` is your friend

In [2]:
# TODO argparse example, lift from Optum code

- argparse is yet another form of documentation

### And so are `screen` and `tmux`

- much better than running python ... (especially better than running a computation in jupyter)
- more sophisticated way of scheduling jobs in the background

### Consider `make` for complex workflows

In [ ]:
# TODO Makefile example, perhaps lifted from CIS 501

## Reproducibility

1. Parameterize everything, including your environment
2. A/B test
4. Consider Docker for complex environments

### Bash scripts as documentation

- Our form of an experimental procedure

### Gives us a mechanismf

## Thoughts on Jupyter Notebooks

## My workflow (still a work in progress)

## References